In [ ]:
# feature engineering
# Conny Lin | June 14, 2020

In [ ]:
# updated June 14, 2020
# computer dependent local variable settings 
import socket, os
# check which computer this code is running on
hostname = socket.gethostname().split('.')[0]
# set local path settings based on computer host
if hostname == 'PFC':
    pylibrary = ['/Users/connylin/Dropbox/Code/proj/']
    pCapstone = '/Users/connylin/Dropbox/CA/ED _20200119 Brain Station Data Science Diploma/Capstone'
    datapath = os.path.join(pCapstone,'data','nutcracker_sample_1Meach.csv')
elif hostname == 'Angular Gyrus':
    pylibrary = ['/Users/connylin/Code/proj/']
    pCapstone = '/Users/connylin/Dropbox/CA/ED _20200119 Brain Station Data Science Diploma/Capstone'
    datapath = os.path.join(pCapstone,'data','nutcracker_sample_1Meach.csv')
else:
    assert False, 'host computer not regonized'
    
# import standard libraries
import sys, pickle, socket
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# load local libraries
for path in pylibrary:
    if path not in sys.path:
        sys.path.insert(1, path)
import brainstation_capstone
from brainstation_capstone.etl.datatransform import Nutcracker

# get machine learning input data
NC = Nutcracker(datapath)
X_train, X_test, y_train, y_test = NC.mldata()

In [ ]:
# ML specific 
outputfolder = os.path.join(pCapstone,'ml_feature_eng')
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression

Many of the features in nutcracker are calculated to represent similar things

In [ ]:
from brainstation_capstone.db import chor_legend


time measures:
* time
* persistence

size measures:
* area

width measures:
* midline
* morphwidth
* width
* relwidth

length measures:
* length
* rellength

angle measures:
* aspect
* relaspect
* kink
* curve
* orient

movement speed measures:
* speed
* crab
* angular

movement dir measures: 
* bias
* dir
* vel_x
* vel_y



In [19]:
# create reference table
le = chor_legend
le

<module 'brainstation_capstone.db.chor_legend' from '/Users/connylin/Dropbox/Code/proj/brainstation_capstone/db/chor_legend.py'>

use OLS 

In [13]:
# list features
X_columns = np.array(NC.names['X']).transpose()

In [14]:
# OLS to view the best features
C=0.05
random_state = 318
max_iter=1000
OLS = LogisticRegression(C=C, random_state=random_state, max_iter=max_iter)
OLS.fit(X_train, y_train)
# take a look at predictive value of each parameter
# set dataframe
top_features = pd.DataFrame(X_columns, columns=['features'])
# get coefficient
X_logistic_coef = OLS.coef_
# put int OLS_coef
top_features['OLS_coef'] = X_logistic_coef[0]
# put in absolute
top_features['OLS_coef_abs'] = np.absolute(top_features['OLS_coef'].values)
top_features.sort_values(ascending=False, inplace=True, by=['OLS_coef_abs'])
top_features

/Users/connylin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,features,OLS_coef,OLS_coef_abs
5,width,-13.761015,13.761015
9,aspect,-10.941442,10.941442
6,relwidth,3.617541,3.617541
10,relaspect,2.827076,2.827076
8,rellength,-2.393815,2.393815
4,morphwidth,2.076456,2.076456
7,length,2.056058,2.056058
2,area,-0.712834,0.712834
13,speed,0.684202,0.684202
20,crab,-0.585187,0.585187


Use variance inflation factor to see

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# get data frame 
df = NC.data()
variance_inflation_factor(X.values, 0)

In [ ]:
pd.Series([variance_inflation_factor(X.values, i) 
               for i in range(X.shape[1])], 
              index=X.columns)

use variance threshold

In [14]:
# get dataframe
df = pd.DataFrame(X_train, columns=NC.names['X'])
# get some idea about variance
df.var()
# Instantiate the VarianceThresholder 
my_vt = VarianceThreshold(threshold=var_threshold)



In [16]:
# Fit to the data and calculate the variances per column
my_vt.fit(df)

# Extract the variances per column
column_variances = my_vt.variances_.round(2)

# Plot with the threshold
plt.figure()
plt.barh(np.flip(df.columns), np.flip(column_variances))
plt.xlabel('Variance ($)')
plt.ylabel('Feature')
plt.axvline(10000, color='black', linestyle='--')
plt.show()

time            683.353613
persistence    7873.363444
area              0.001777
midline           0.037433
morphwidth        0.000944
width             0.008478
relwidth          0.097917
length            0.041411
rellength         0.013175
aspect            0.015791
relaspect         0.160896
kink            595.379829
curve           100.540799
speed             0.013098
angular         194.503831
bias              0.295370
dir               0.003717
vel_x             0.022247
vel_y             0.022225
orient         9826.781548
crab              0.000356
dtype: float64

ValueError: No feature in X meets the variance threshold 10000.00000